Copyright 2021-2024 Lawrence Livermore National Security, LLC and other MuyGPyS
Project Developers. See the top-level COPYRIGHT file for details.

SPDX-License-Identifier: MIT

# Anisotropic Metric Tutorial

This notebook walks through a simple anisotropic regression workflow and illustrates anisotropic features of `MuyGPyS`.

In [ ]:
import sys
for m in sys.modules.keys():
    if m.startswith("Muy"):
        sys.modules.pop(m)
%env MUYGPYS_BACKEND=numpy
%env MUYGPYS_FTYPE=64

In [ ]:
import numpy as np

from MuyGPyS._test.sampler import UnivariateSampler2D, print_results
from MuyGPyS.gp import MuyGPS
from MuyGPyS.gp.deformation import Anisotropy, Isotropy, l2
from MuyGPyS.gp.hyperparameter import AnalyticScale, Parameter, VectorParameter
from MuyGPyS.gp.kernels import Matern
from MuyGPyS.gp.noise import HomoscedasticNoise
from MuyGPyS.neighbors import NN_Wrapper
from MuyGPyS.optimize import Bayes_optimize
from MuyGPyS.optimize.batch import sample_batch
from MuyGPyS.optimize.loss import lool_fn

We will set a random seed here for consistency when building docs.
In practice we would not fix a seed.

In [ ]:
np.random.seed(0)

## Sampling a 2D Surface from a Conventional GP

This notebook will use a simple two-dimensional curve sampled from a conventional Gaussian process.
We will specify the domain as a simple grid on a one-dimensional surface and divide the observations näively into train and test data.

Feel free to download the source notebook and experiment with different parameters.

First we specify the data size and the proportion of the train/test split.

In [ ]:
points_per_dim = 60
train_ratio = 0.05

We will assume that the true data is produced with no noise, so we specify a very small noise prior for numerical stability.
This is an idealized experiment with effectively no instrument error.

In [ ]:
nugget_noise = HomoscedasticNoise(1e-14)

We will perturb our simulated observations (the training data) with some i.i.d Gaussian measurement noise.

In [ ]:
measurement_noise = HomoscedasticNoise(1e-7)

Finally, we will specify a Matérn kernel with hyperparameters.
`smoothness` determines how differentiable the GP prior is.
The larger `smoothness` grows, the smoother sampled functions will become.

In [ ]:
sim_smoothness = Parameter(1.5)

We will use an anisotropic deformation, where displacement along the dimensions are weighted differently.
Each dimension has a corresponding `length_scale` parameter.

In [ ]:
sim_length_scale0 = Parameter(0.1)
sim_length_scale1 = Parameter(0.5)

We use all of these parameters to define a Matérn kernel GP and a sampler for convenience.
The `UnivariateSampler2D` class is a convenience class for this tutorial, and is not a part of the library.
We will use an anisotropic deformation to ensure that we sample data from the appropriate distribution.

In [ ]:
sampler = UnivariateSampler2D(
    points_per_dim=points_per_dim,
    train_ratio=train_ratio,
    kernel=Matern(
        smoothness=sim_smoothness,
        deformation=Anisotropy(
            l2,
            length_scale=VectorParameter(
                sim_length_scale0,
                sim_length_scale1,
            ),
        ),
    ),
    noise=nugget_noise,
    measurement_noise=measurement_noise,
)

Finally, we will sample a curve from this GP prior and visualize it.
Note that we perturb the train responses (the values that our model will actual receive) with Gaussian measurement noise.
Further note that this is not especially fast, as sampling from a conventional Gaussian process requires computing the Cholesky decomposition of a `(data_count, data_count)` matrix.

In [ ]:
train_features, test_features = sampler.features()

In [ ]:
train_responses, test_responses = sampler.sample()

In [ ]:
sampler.plot_sample()

We can observe that our choice of anisotropy has caused the globular Gaussian features in the sampled surface to "smear" in the direction of the more heavily weighted axis.  

## Training an Anisotropic Model

We will not belabor the details covered in the [Univariate Regression Tutorial](./univariate_regression_tutorial.ipynb).
We must similarly construct a nearest neighbors index and sample a training batch in order to optimize a model.

⚠️ For now, we use isotropic nearest neighbors as we do not have a guess as to the anisotropic scaling. Future versions of the library will use learned anisotropy to modify neighborhood structure during optimization. ⚠️

In [ ]:
nn_count = 30
nbrs_lookup = NN_Wrapper(train_features, nn_count, nn_method="exact", algorithm="ball_tree")
batch_count = sampler.train_count
batch_indices, batch_nn_indices = sample_batch(
    nbrs_lookup, batch_count, sampler.train_count
)

We construct a MuyGPs object with a Matérn kernel.
For simplicity, we will fix `smoothness` and attempt to optimize the two `length_scale` parameters.

In [ ]:
muygps_anisotropic = MuyGPS(
    kernel=Matern(
        smoothness=sim_smoothness,
        deformation=Anisotropy(
            l2,
            length_scale=VectorParameter(
                Parameter("log_sample", (0.01, 1.0)),
                Parameter("log_sample", (0.01, 1.0)),
            ),
        ),
    ),
    noise=measurement_noise,
    scale=AnalyticScale(),
)

We will also create and optimze an isotropic model for comparison.

In [ ]:
muygps_isotropic = MuyGPS(
    kernel=Matern(
        smoothness=sim_smoothness,
        deformation=Isotropy(
            l2,
            length_scale=Parameter("log_sample", (0.01, 1.0)),
        ),
    ),
    noise=measurement_noise,
)

We build our difference tensors as usual and use Bayesian optimization.
Note that there is a difference between the crosswise and pairwise tensors that we create here, versus those we create for an isotropic kernel.
Anisotropic models create _difference_ tensors rather than _distance_ tensors, which have an extra dimension recording the feature dimension-wise comparisons (in this case, differences) between the items being compared.
This is an important distinction, as anisotropic models need to record feature-dimension-wise comparisons to be scaled by trainable parameters, whereas isotropic models do not and collapse differences directory into distances.

In [ ]:
(
    batch_crosswise_diffs,
    batch_pairwise_diffs,
    batch_targets,
    batch_nn_targets,
) = muygps_anisotropic.make_train_tensors(
    batch_indices,
    batch_nn_indices,
    train_features,
    train_responses,
)

Keyword arguments for the optimization:

In [ ]:
opt_kwargs = {
    "loss_fn": lool_fn,
    "verbose": True,
    "random_state": 1,
    "init_points": 5,
    "n_iter": 30,
    "allow_duplicate_points": True,
}

In [ ]:
muygps_anisotropic = Bayes_optimize(
    muygps_anisotropic,
    batch_targets,
    batch_nn_targets,
    batch_crosswise_diffs,
    batch_pairwise_diffs,
    **opt_kwargs,
)

In [ ]:
print(f"BayesianOptimization finds an optimimal pair of length scales: {muygps_anisotropic.kernel.deformation.length_scale()}")

Note here that these returned length scale values might be a little different than what we used to sample the surface.
This can be due to a few factors:
1. optimizer might not have run enough iterations to converge, or
2. there is some mutual unidentifiability between the length scale parameters and the variance `scale` parameter.

However, `length_scale0 < length_scale1` as expected.

We also optimize the isotropic benchmark.
Notice that we need to construct new _distance_ tensors for the isotropic model.

In [ ]:
(
    batch_crosswise_dists,
    batch_pairwise_dists,
    _,
    _,
) = muygps_isotropic.make_train_tensors(
    batch_indices,
    batch_nn_indices,
    train_features,
    train_responses,
)

In [ ]:
muygps_isotropic = Bayes_optimize(
    muygps_isotropic,
    batch_targets,
    batch_nn_targets,
    batch_crosswise_dists,
    batch_pairwise_dists,
    **opt_kwargs,
)

In [ ]:
print(f"BayesianOptimization finds that the optimimal isotropic length scale is {muygps_isotropic.kernel.deformation.length_scale()}")

We see here that when fixed to an isotropic length scale, Bayesian optimization tends to favor the smallest true length scale.
We'll see how this affects modeling, prediction, and uncertainty quanlity below.

We separately optimize the `scale` variance scale parameter for each model.

In [ ]:
muygps_anisotropic = muygps_anisotropic.optimize_scale(
    batch_pairwise_diffs, batch_nn_targets
)
muygps_isotropic = muygps_isotropic.optimize_scale(
    batch_pairwise_diffs, batch_nn_targets
)

## Inference

As in the [Univariate Regression Tutorial](./univariate_regression_tutorial.ipynb), we must realize difference tensors formed from the testing data and apply them to form Gaussian process predictions for our problem.

In [ ]:
test_count, _ = test_features.shape
indices = np.arange(test_count)
test_nn_indices, _ = nbrs_lookup.get_nns(test_features)
(
    test_crosswise_diffs,
    test_pairwise_diffs,
    test_nn_targets,
) = muygps_anisotropic.make_predict_tensors(
    indices,
    test_nn_indices,
    test_features,
    train_features,
    train_responses,
)
(
    test_crosswise_dists,
    test_pairwise_dists,
    _,
) = muygps_isotropic.make_predict_tensors(
    indices,
    test_nn_indices,
    test_features,
    train_features,
    train_responses,
)

As in the [Univariate Regression Tutorial](univariate_regression_tutorial.ipynb) we will evaluate the prediction performance of our models in terms of RMSE, mean diagonal posterior variance, the mean 95% confidence interval size, and the coverage, which ideally should be near 95%.

In [ ]:
Kcross_anisotropic = muygps_anisotropic.kernel(test_crosswise_diffs)
Kin_anisotropic = muygps_anisotropic.kernel(test_pairwise_diffs)

predictions_anisotropic = muygps_anisotropic.posterior_mean(
    Kin_anisotropic, Kcross_anisotropic, test_nn_targets
)
variances_anisotropic = muygps_anisotropic.posterior_variance(
    Kin_anisotropic, Kcross_anisotropic
)
confidence_intervals_anisotropic = np.sqrt(variances_anisotropic) * 1.96
coverage_anisotropic = (
    np.count_nonzero(
        np.abs(test_responses - predictions_anisotropic) < confidence_intervals_anisotropic
    ) / test_count
)

We also evaluate the isotropic model

In [ ]:
Kcross_isotropic = muygps_isotropic.kernel(test_crosswise_dists)
Kin_isotropic = muygps_isotropic.kernel(test_pairwise_dists)

predictions_isotropic = muygps_isotropic.posterior_mean(Kin_isotropic, Kcross_isotropic, test_nn_targets)
variances_isotropic = muygps_isotropic.posterior_variance(Kin_isotropic, Kcross_isotropic)

confidence_intervals_isotropic = np.sqrt(variances_isotropic) * 1.96
coverage_isotropic = (
    np.count_nonzero(
        np.abs(test_responses - predictions_isotropic) < confidence_intervals_isotropic
    ) / test_count
)

## Results comparison

A comparison of our trained models reveals that the anisotropic kernel gets close to the true `(0.1, 0.5)` length scale, whereas the isotropic model has to learn a single parameter that has to split the difference somehow.
This results in both a higher RMSE and larger confidence intervals in order to achieve similar coverage.

In [ ]:
print_results(
    test_responses,
    ("anisotropic", muygps_anisotropic, predictions_anisotropic, variances_anisotropic, confidence_intervals_anisotropic, coverage_anisotropic),
    ("isotropic", muygps_isotropic, predictions_isotropic, variances_isotropic, confidence_intervals_isotropic, coverage_isotropic),
)

This dataset is low-dimensional so we can plot our predictions and visually evaluate their performance. 
We plot below the expected (true) surface, and the surface that our model predicts.
Note that they are visually similar and major trends are captured, although there are some differences. 

In [ ]:
sampler.plot_predictions(("Anisotropic", predictions_anisotropic), ("Isotropic", predictions_isotropic))

As we can see, the anisotropic model learns a surface that is much visually closer to what is expected.
In particular, the isotropic surface has blobby circular features as it to be expected, as it is unable to differentiate between distances along the different axes.

We will also investigate more details information about the errors.
Below we produce three plots that help us to understand our results.
The left plot shows the residual, which is the difference between the true values and our expectations.
The middle plot shows the magnitude of the 95% confidence interval.
The larger the confidence interval, the less certain the model is of its predictions.
Finally, the right plot shows the difference between the 95% confidence interval length and the magnitude of the residual.
All of the points larger than zero (in red) are not captured by the confidence interval.
Hence, this plot shows our coverage distribution.

In [ ]:
sampler.plot_errors(
    ("Anisotropic", predictions_anisotropic, confidence_intervals_anisotropic),
    ("Isotropic", predictions_isotropic, confidence_intervals_isotropic),
)

The rightmost columns shows that the anisotropic assumptions both obtains lower residuals, i.e. the posterior means are more accurate.
The middle column shows that the the posterior variances (and resulting confidence intervals) are smaller, and therefore the anisotropic model is also more confident in its predictions.
Finally, the rightmost plot reveals the uncovered points - all red-scale residuals exceed the confidence interval.
Not only does the isotropic model appear to have more uncovered points, they tend to be further outside of the confidence interval than those of the anisotropic model.
These results demonstrate the importance of correct model assumptions, both on predictions and uncertainty quantification.